In [34]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=a8efd64881a9436c257eaa8efb69ec5adfc5b087db93a9804a1c59bb439cd8ce
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [21]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd

file_path = "/content/drive/MyDrive/final_reviews_sentiment.csv"

# Load CSV with error handling, skip bad lines
try:
    df = pd.read_csv(file_path, quotechar='"', escapechar='\\', error_bad_lines=False)
    print(f"Data loaded with shape: {df.shape}")
except Exception as e:
    print(f"Error loading CSV: {e}")

Error loading CSV: read_csv() got an unexpected keyword argument 'error_bad_lines'


In [23]:
file_path = '/content/drive/MyDrive/final_reviews_sentiment.csv'
df = pd.read_csv(file_path)


In [24]:
# Display the first few rows of the data
df.head()

# Optional: Check for any missing values
df.isnull().sum()

# Check data info (column types, non-null counts)
df.info()

# Display the basic statistics of numeric columns
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   business_id      1000000 non-null  object 
 1   cool             1000000 non-null  int64  
 2   date             1000000 non-null  object 
 3   funny            1000000 non-null  int64  
 4   review_id        1000000 non-null  object 
 5   stars            1000000 non-null  float64
 6   text             1000000 non-null  object 
 7   useful           1000000 non-null  int64  
 8   user_id          1000000 non-null  object 
 9   review_length    1000000 non-null  int64  
 10  review_year      1000000 non-null  int64  
 11  review_month     1000000 non-null  int64  
 12  rating_category  1000000 non-null  object 
 13  sentiment        1000000 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 106.8+ MB


,cool,funny,stars,useful,review_length,review_year,review_month
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,0.488093,0.315746,3.758164,1.151757,100.084665,2016.539016,6.449989
std,2.045129,1.610770,1.474333,3.256298,83.155012,3.077030,3.401219
min,-1.000000,-1.000000,1.000000,-1.000000,1.000000,2005.000000,1.000000
25%,0.000000,0.000000,3.000000,0.000000,41.000000,2015.000000,3.000000
50%,0.000000,0.000000,4.000000,0.000000,74.000000,2017.000000,7.000000
75%,0.000000,0.000000,5.000000,1.000000,131.000000,2019.000000,9.000000
max,359.000000,369.000000,5.000000,1182.000000,512.000000,2022.000000,12.000000


**Sentiment Distribution - Understand the overall distribution of customer sentiments across the dataset**

In [35]:
import pandasql as psql

# SQL query
query = """
    SELECT sentiment, COUNT(*) AS sentiment_count
    FROM df
    GROUP BY sentiment
    ORDER BY sentiment_count DESC
"""

# Execute the query
result = psql.sqldf(query, locals())

# Display the result
print(result)



  sentiment  sentiment_count
0  POSITIVE           691822
1  NEGATIVE           308178


**Sentiment by Rating Category (Stars) - Analyze the sentiment distribution across different star ratings (1-5 stars).**

In [36]:
# SQL query to group by rating_category and sentiment, and count the sentiment
query = """
    SELECT rating_category, sentiment, COUNT(*) AS sentiment_count
    FROM df
    GROUP BY rating_category, sentiment
    ORDER BY rating_category, sentiment_count DESC
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)


  rating_category sentiment  sentiment_count
0            high  POSITIVE           630449
1            high  NEGATIVE            42469
2             low  NEGATIVE           214114
3             low  POSITIVE            14237
4          medium  NEGATIVE            51595
5          medium  POSITIVE            47136


Average Rating by Sentiment - Calculate the average rating for each sentiment (positive, negative, neutral).

In [38]:
query = """
    SELECT sentiment, AVG(stars) AS avg_rating
    FROM df
    GROUP BY sentiment
    ORDER BY avg_rating DESC;
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

  sentiment  avg_rating
0  POSITIVE    4.529606
1  NEGATIVE    2.026371


 Reviews Per Year/Month by Sentiment - Track sentiment over time and identify trends in customer satisfaction.

In [39]:
# SQL query to group by review_year and sentiment, and count the sentiment
query = """
    SELECT review_year, sentiment, COUNT(*) AS sentiment_count
    FROM df
    GROUP BY review_year, sentiment
    ORDER BY review_year, sentiment_count DESC;
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

    review_year sentiment  sentiment_count
0          2005  POSITIVE               89
1          2005  NEGATIVE               29
2          2006  POSITIVE              409
3          2006  NEGATIVE              129
4          2007  POSITIVE             1718
5          2007  NEGATIVE              599
6          2008  POSITIVE             5067
7          2008  NEGATIVE             1850
8          2009  POSITIVE             7530
9          2009  NEGATIVE             3004
10         2010  POSITIVE            14140
11         2010  NEGATIVE             5540
12         2011  POSITIVE            23584
13         2011  NEGATIVE             9573
14         2012  POSITIVE            28741
15         2012  NEGATIVE            12124
16         2013  POSITIVE            38418
17         2013  NEGATIVE            16625
18         2014  POSITIVE            52210
19         2014  NEGATIVE            22202
20         2015  POSITIVE            68614
21         2015  NEGATIVE            30002
22         

Sentiment by Review Length

In [40]:
# SQL query to calculate average review length grouped by sentiment
query = """
    SELECT sentiment, AVG(review_length) AS avg_review_length
    FROM df
    GROUP BY sentiment
    ORDER BY avg_review_length DESC;
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

  sentiment  avg_review_length
0  NEGATIVE         125.565663
1  POSITIVE          88.733937


Sentiment Distribution by Business Category

In [41]:
# SQL query to count sentiments for each business and sentiment combination
query = """
    SELECT business_id, sentiment, COUNT(*) AS sentiment_count
    FROM df
    GROUP BY business_id, sentiment
    ORDER BY sentiment_count DESC;
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

                   business_id sentiment  sentiment_count
0       ac1AeYqs8Z4_e2X5M3if2A  POSITIVE              867
1       _ab50qdWOk0DdB6XOrBitw  POSITIVE              831
2       GXFMD0Z4jEVZBCsbPf4CTQ  POSITIVE              733
3       ytynqOUb3hjKeJfRj5Tshw  POSITIVE              698
4       oBNrLz4EDhiscSlbOl8uAw  POSITIVE              629
...                        ...       ...              ...
191632  zziDpuuJw-Km1J4BaGpBKA  POSITIVE                1
191633  zzjCxn89a7RQo8keIOO_Ag  NEGATIVE                1
191634  zznJox6-nmXlGYNWgTDwQQ  POSITIVE                1
191635  zzw66H6hVjXQEt0Js3Mo4A  NEGATIVE                1
191636  zzw66H6hVjXQEt0Js3Mo4A  POSITIVE                1

[191637 rows x 3 columns]


Most Frequent Words in Positive vs Negative Reviews

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to get most common words for each sentiment
def most_common_words(sentiment):
    reviews = df[df['sentiment'] == sentiment]['text']
    X = vectorizer.fit_transform(reviews)
    words = vectorizer.get_feature_names_out()
    word_count = X.toarray().sum(axis=0)
    word_freq = list(zip(words, word_count))
    word_freq_sorted = sorted(word_freq, key=lambda x: x[1], reverse=True)
    return word_freq_sorted

# Get most frequent words for positive and negative sentiments
positive_words = most_common_words('POSITIVE')
negative_words = most_common_words('NEGATIVE')

# Create DataFrames for the words
positive_df = pd.DataFrame(positive_words, columns=['word', 'count'])
positive_df['sentiment'] = 'POSITIVE'

negative_df = pd.DataFrame(negative_words, columns=['word', 'count'])
negative_df['sentiment'] = 'NEGATIVE'

# Combine the positive and negative DataFrames
combined_df = pd.concat([positive_df, negative_df], ignore_index=True)

# Query using pandasql
query = """
    SELECT sentiment, word, count
    FROM combined_df
    ORDER BY sentiment, count DESC
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)


   sentiment        word   count
0   NEGATIVE        food  171561
1   NEGATIVE        just  138500
2   NEGATIVE       place  132439
3   NEGATIVE        like  125084
4   NEGATIVE        time  123062
5   NEGATIVE        good  121835
6   NEGATIVE     service  121215
7   NEGATIVE       order   81166
8   NEGATIVE         got   78629
9   NEGATIVE         don   78528
10  NEGATIVE        didn   74641
11  NEGATIVE        said   71480
12  NEGATIVE     ordered   71062
13  NEGATIVE         did   68880
14  NEGATIVE      really   66855
15  NEGATIVE        told   65528
16  NEGATIVE        came   63452
17  NEGATIVE        went   60560
18  NEGATIVE       great   59264
19  NEGATIVE     minutes   58733
20  POSITIVE       great  377380
21  POSITIVE        good  343895
22  POSITIVE        food  340687
23  POSITIVE       place  315552
24  POSITIVE     service  204758
25  POSITIVE        time  187673
26  POSITIVE        like  173540
27  POSITIVE        just  171561
28  POSITIVE      really  155129
29  POSITI

Top 10 Reviews with Most 'Useful' Votes

In [43]:
# SQL query to select review_id, text, useful and order by 'useful' in descending order
query = """
    SELECT review_id, text, useful
    FROM df
    ORDER BY useful DESC
    LIMIT 10
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

                review_id                                               text  \
0  b0HBLNn02tkpC9rIyPRc_Q  Avoid at ALL costs.  Do NOT do business with D...   
1  kTSpD7akIapty9AtLwJ11g  WORST COMPANY EVER!\n\nI had the unfortunate e...   
2  KqMVXzlSMv7jRvt_raknMQ  Ultra upscale restaurant with outstanding food...   
3  ZVlZ5CGSKE4gnpAIeSJMvg  Well executed restaurant in Reno. Very modern ...   
4  IaIAHPgq1DCGlix_83fSlQ  A disappointment. Our server was charming, but...   
5  DQhraLAoQaMND019yGr-zg  This place is one of the top 3 best restaurant...   
6  NhiseboRNItRx2_Cg2hejw  Good food, OK wine list, horrid service; will ...   
7  dYwpj-RSnXUAfJJXhBZMHg  Really hate to do this, since our past experie...   
8  KvET6SuK_a9-45iLwAaO3g  I have been to this store several times over t...   
9  R_l6aeULCXZ_Ww0H_XlWOQ  Now that everyone got to see me rocking my bra...   

   useful  
0    1182  
1     997  
2     359  
3     358  
4     332  
5     260  
6     240  
7     226  
8     225  

Top Businesses with Most Positive Sentiment

In [47]:
# SQL query to select business_id and count the number of positive reviews
query = """
    SELECT business_id, COUNT(*) AS positive_review_count
    FROM df
    WHERE sentiment = 'POSITIVE'
    GROUP BY business_id
    ORDER BY positive_review_count DESC
    LIMIT 10
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

              business_id  positive_review_count
0  ac1AeYqs8Z4_e2X5M3if2A                    867
1  _ab50qdWOk0DdB6XOrBitw                    831
2  GXFMD0Z4jEVZBCsbPf4CTQ                    733
3  ytynqOUb3hjKeJfRj5Tshw                    698
4  oBNrLz4EDhiscSlbOl8uAw                    629
5  _C7QiQQc47AOEv4PE3Kong                    582
6  GBTPC53ZrG1ZBY3DT8Mbcw                    555
7  VQcCL9PiNL_wkGf-uF3fjg                    531
8  gTC8IQ_i8zXytWSly3Ttvg                    521
9  6a4gLLFSgr-Q6CZXDLzBGQ                    508


Reviews with the Most 'Cool', 'Funny', or 'Useful' Votes

In [45]:
# SQL query to select review_id, text, cool, funny, useful, and order by 'useful' in descending order
query = """
    SELECT review_id, text, cool, funny, useful
    FROM df
    ORDER BY useful DESC
    LIMIT 10
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

                review_id                                               text  \
0  b0HBLNn02tkpC9rIyPRc_Q  Avoid at ALL costs.  Do NOT do business with D...   
1  kTSpD7akIapty9AtLwJ11g  WORST COMPANY EVER!\n\nI had the unfortunate e...   
2  KqMVXzlSMv7jRvt_raknMQ  Ultra upscale restaurant with outstanding food...   
3  ZVlZ5CGSKE4gnpAIeSJMvg  Well executed restaurant in Reno. Very modern ...   
4  IaIAHPgq1DCGlix_83fSlQ  A disappointment. Our server was charming, but...   
5  DQhraLAoQaMND019yGr-zg  This place is one of the top 3 best restaurant...   
6  NhiseboRNItRx2_Cg2hejw  Good food, OK wine list, horrid service; will ...   
7  dYwpj-RSnXUAfJJXhBZMHg  Really hate to do this, since our past experie...   
8  KvET6SuK_a9-45iLwAaO3g  I have been to this store several times over t...   
9  R_l6aeULCXZ_Ww0H_XlWOQ  Now that everyone got to see me rocking my bra...   

   cool  funny  useful  
0     0      0    1182  
1     0      0     997  
2   359      1     359  
3   358      0     

Top 10 Businesses with Most Negative Sentiment

In [46]:
# SQL query to select business_id and count the number of negative reviews
query = """
    SELECT business_id, COUNT(*) AS negative_review_count
    FROM df
    WHERE sentiment = 'NEGATIVE'
    GROUP BY business_id
    ORDER BY negative_review_count DESC
    LIMIT 10
"""

# Execute the query using pandasql
result = psql.sqldf(query, locals())

# Display the result
print(result)

              business_id  negative_review_count
0  PP3BBaVxZLcJU54uP_wL6Q                    290
1  iSRTaT9WngzB8JJ2YKJUig                    276
2  IkY2ticzHEn4QFn8hQLSWg                    269
3  wz8ZPfySQczcPgSyd33-HQ                    261
4  _ab50qdWOk0DdB6XOrBitw                    260
5  ac1AeYqs8Z4_e2X5M3if2A                    205
6  -QI8Qi8XWH3D8y8ethnajA                    196
7  VQcCL9PiNL_wkGf-uF3fjg                    193
8  ve_-rPlEGPClytyJtY55Vw                    183
9  Eb1XmmLWyt_way5NNZ7-Pw                    167
